### Cash-flow simulation

In [191]:
import numpy as np
import pandas as pd

def simulate_private_equity_cashflows(
    n_funds=2400,
    quarters_per_fund=60,
    random_seed=11
):
    np.random.seed(random_seed)
    
    # 1. Create fund universe
    funds = []
    for i in range(n_funds):
        vintage = 2000 + i % 20  # 20 vintages
        strategy = 'Buyout' if (i % 2) == 0 else 'Venture'
        geography = 'North America' if (i % 4) < 2 else 'Rest of World'
        quality = 'Good' if (i % 8) < 4 else 'Bad'
        funds.append({
            'FundID': i,
            'VintageYear': vintage,
            'Strategy': strategy,
            'Geography': geography,
            'FundQuality': quality
        })
    funds_df = pd.DataFrame(funds)
    
    # 2. Generate cash flows for each fund
    cashflow_dfs = []
    
    for idx, fund in funds_df.iterrows():
        quarters = np.arange(0, quarters_per_fund)

        # Contributions (capital calls)
        contrib_periods = np.random.choice(quarters[:12], size=4, replace=False)
        contrib_cashflows = np.random.uniform(-0.2, -0.05, size=4)

        # Distributions (returns)
        dist_start = np.random.randint(12, 24)
        dist_quarters = np.random.choice(quarters[dist_start:], size=10, replace=False)
        dist_cashflows = np.random.uniform(0.02, 0.08, size=10)

        # Initialize cashflows
        cashflow_series = np.zeros(quarters_per_fund)
        cashflow_series[contrib_periods] = contrib_cashflows
        cashflow_series[dist_quarters] = dist_cashflows

        # Apply adjustments based on fund characteristics
        if fund['Strategy'] == 'Venture':
            cashflow_series *= 1.3
        if fund['Geography'] == 'North America':
            cashflow_series *= 1.1
        else:
            cashflow_series *= 0.9
        if fund['VintageYear'] % 2 == 1:
            cashflow_series *= 1.1
        else:
            cashflow_series *= 0.9
        if fund['FundQuality'] == 'Good':
            cashflow_series *= 1.1
        else:
            cashflow_series *= 0.9

        cf_df = pd.DataFrame({
            'FundID': fund['FundID'],
            'VintageYear': fund['VintageYear'],
            'Strategy': fund['Strategy'],
            'Geography': fund['Geography'],
            'FundQuality': fund['FundQuality'],
            'Quarter': quarters,
            'Cashflow': cashflow_series
        })

        cashflow_dfs.append(cf_df)

    # 3. Merge everything into one big tidy DataFrame
    all_cashflows_df = pd.concat(cashflow_dfs, ignore_index=True)
    
    return all_cashflows_df


In [192]:
cashflows = simulate_private_equity_cashflows()
cashflows.head(10)

,FundID,VintageYear,Strategy,Geography,FundQuality,Quarter,Cashflow
0,0,2000,Buyout,North America,Good,0,0.000000
1,0,2000,Buyout,North America,Good,1,0.000000
2,0,2000,Buyout,North America,Good,2,0.000000
3,0,2000,Buyout,North America,Good,3,-0.063956
4,0,2000,Buyout,North America,Good,4,0.000000
5,0,2000,Buyout,North America,Good,5,0.000000
6,0,2000,Buyout,North America,Good,6,-0.138188
7,0,2000,Buyout,North America,Good,7,0.000000
8,0,2000,Buyout,North America,Good,8,-0.078823
9,0,2000,Buyout,North America,Good,9,0.000000


In [193]:
cashflows_wide = cashflows.pivot(index='FundID', columns='Quarter', values='Cashflow')
cashflows_wide.T.head()

FundID,0,1,2,3,4,5,6,7,8,9,...,2390,2391,2392,2393,2394,2395,2396,2397,2398,2399
Quarter,,,,,,,,,,,,,,,,,,,,,
0,0.000000,0.000000,-0.083402,0.000000,0.000000,-0.158611,0.000000,-0.212671,-0.123591,0.000000,...,-0.093344,0.0,-0.089233,0.0,-0.133613,0.000000,0.000000,-0.271889,-0.039558,-0.119876
1,0.000000,-0.099205,0.000000,0.000000,-0.144423,0.000000,0.000000,0.000000,0.000000,-0.102610,...,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,-0.114178,0.000000,-0.257803,0.000000,-0.128033,0.000000,0.000000,...,0.000000,0.0,0.000000,0.0,0.000000,-0.176276,-0.138173,-0.115097,0.000000,-0.196255
3,-0.063956,0.000000,0.000000,0.000000,0.000000,0.000000,-0.068164,0.000000,-0.077769,-0.296914,...,-0.137558,0.0,0.000000,0.0,-0.081990,0.000000,0.000000,0.000000,-0.120399,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,-0.083656,0.0,0.000000,0.000000,0.000000,-0.175449,0.000000,0.000000


### Example of IRR calculation on portfolio of funds

In [194]:
pip install numpy-financial

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\Asus\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [195]:
from numpy_financial import irr 
def compute_irr(cashflows): 
    return irr(cashflows)


In [196]:
def moic(cashflows): 
    return cashflows[cashflows > 0].sum() / abs(cashflows[cashflows < 0].sum()) if cashflows[cashflows < 0].sum() != 0 else np.nan

In [197]:
def simulate_portfolio_ret(cashflows_df, portfolio_size=50, random_seed=None):
    if random_seed is not None:
        np.random.seed(random_seed)

    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows_df['FundID'].unique(), size=portfolio_size, replace=False)

    # Filter the cashflows for the selected portfolio
    portfolio_df = cashflows_df[cashflows_df['FundID'].isin(selected_funds)]

    # Aggregate cash flows per quarter
    aggregated_cf = (
        portfolio_df.groupby('Quarter')['Cashflow']
        .sum()
        .sort_index()
        .values
    )

    # Compute IRR
    portfolio_irr = compute_irr(aggregated_cf)
    portfolio_moic = moic(aggregated_cf)

    return {
        'PortfolioFunds': selected_funds,
        'IRR': portfolio_irr,
        'Cashflows': aggregated_cf,
        'MOIC': portfolio_moic,
        'Portfolio_df': portfolio_df,
    }


In [198]:
# Simulate the full fund universe
cashflows_df = simulate_private_equity_cashflows()

# Simulate one portfolio
portfolio_result = simulate_portfolio_ret(cashflows_df, portfolio_size=500, random_seed=5)

print(f"Portfolio IRR: {portfolio_result['IRR']:.2%}")
print(f"Portfolio MOIC: {portfolio_result['MOIC']:.2f}")


Portfolio IRR: 0.02%
Portfolio MOIC: 1.01


### Attribution

In [199]:
cashflows_df = simulate_private_equity_cashflows()
portfolio_result = simulate_portfolio_ret(cashflows_df, portfolio_size=1000, random_seed=1)

In [200]:
import numpy_financial as npf
def attribution_analysis(cashflows_df, portfolio_result):

    portfolio_funds = portfolio_result['PortfolioFunds']
    portfolio_cf = portfolio_result['Cashflows']
    portfolio_irr = portfolio_result['IRR']
    potrfolio_moic = portfolio_result['MOIC']
    portfolio_df = portfolio_result['Portfolio_df']

    # 1. MARKET Average IRR (baseline)
    market_cf = cashflows_df.groupby('Quarter')['Cashflow'].sum().sort_index().values
    market_irr = npf.irr(market_cf)
    market_moic = moic(market_cf)

    # 2. TIMING Alpha: invest according to portfolio vintages but into the market
    # --> Weight vintages like the portfolio, but performance is market average
    fund_info = cashflows_df[['FundID', 'VintageYear', 'Strategy', 'Geography']].drop_duplicates()

    portfolio_info = fund_info[fund_info['FundID'].isin(portfolio_funds)]

    # Get the market vintage distribution
    invested_df = cashflows_df[cashflows_df['Cashflow'] < 0]
    distr_df = cashflows_df[cashflows_df['Cashflow'] > 0]
    vintage_c = invested_df.groupby('VintageYear')['Cashflow'].sum().abs()
    vintage_d = distr_df.groupby('VintageYear')['Cashflow'].sum()
    vintage_weights_c = invested_df.groupby('VintageYear')['Cashflow'].sum().abs() / abs(invested_df['Cashflow'].sum())
    vintage_weights_d = distr_df.groupby('VintageYear')['Cashflow'].sum() / abs(distr_df['Cashflow'].sum())

    #get the portfolio vintage distribution
    port_invested_df = portfolio_df[portfolio_df['Cashflow'] < 0]
    port_distr_df = portfolio_df[portfolio_df['Cashflow'] > 0]
    vintage_c_port = port_invested_df.groupby('VintageYear')['Cashflow'].sum().abs()
    vintage_d_port = port_distr_df.groupby('VintageYear')['Cashflow'].sum() 
    vintage_weights_c_port = port_invested_df.groupby('VintageYear')['Cashflow'].sum().abs() / abs(port_invested_df['Cashflow'].sum())
    vintage_weights_d_port = port_distr_df.groupby('VintageYear')['Cashflow'].sum() / abs(port_distr_df['Cashflow'].sum())

    # Simulate market cash flows weighted by portfolio's vintage weights
    timing_cf = np.zeros_like(market_cf)
    for vintage, weight in vintage_weights_c_port.items():
        vintage_funds = fund_info[fund_info['VintageYear'] == vintage]['FundID']
        vintage_cf = invested_df[invested_df['FundID'].isin(vintage_funds)].groupby('Quarter')['Cashflow'].sum()
        timing_cf += (weight / vintage_c[vintage]) * vintage_cf.reindex(np.arange(60), fill_value=0).values

    for vintage, weight in vintage_weights_d_port.items():
        vintage_funds = fund_info[fund_info['VintageYear'] == vintage]['FundID']
        vintage_cf = distr_df[distr_df['FundID'].isin(vintage_funds)].groupby('Quarter')['Cashflow'].sum()
        timing_cf += (weight / vintage_d[vintage]) * vintage_cf.reindex(np.arange(60), fill_value=0).values

    timing_irr = npf.irr(timing_cf)
    timing_moic = moic(timing_cf)
   

    # 3. STRATEGY Alpha: allocation across Buyout vs Venture
    invested_grouped = (
        invested_df
        .groupby(['VintageYear', 'Strategy'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )
    distr_grouped = (
        distr_df
        .groupby(['VintageYear', 'Strategy'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )

    port_invested_grouped = (
        port_invested_df
        .groupby(['VintageYear', 'Strategy'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )
    port_distr_grouped = (
        port_distr_df
        .groupby(['VintageYear', 'Strategy'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )

    strategy_cf = np.zeros_like(market_cf)

    for idx, row in invested_grouped.iterrows():
        vintage = row['VintageYear']
        strategy = row['Strategy']
        weight = row['Cashflow']

        vintage_strategy_fund = fund_info[
            (fund_info['VintageYear'] == vintage) &
            (fund_info['Strategy'] == strategy)
        ]['FundID']
        strat_cf = invested_df[invested_df['FundID'].isin(vintage_strategy_fund)].groupby('Quarter')['Cashflow'].sum()
        port_vintage_strategy_weight = port_invested_grouped[(port_invested_grouped['VintageYear'] == vintage) & (port_invested_grouped['Strategy'] == strategy)]['Cashflow'].values[0]
        strategy_cf += (vintage_weights_c[vintage]*port_vintage_strategy_weight / vintage_c_port[vintage] / weight) * strat_cf.reindex(np.arange(60), fill_value=0).values

    for idx, row in distr_grouped.iterrows():
        vintage = row['VintageYear']
        strategy = row['Strategy']
        weight = row['Cashflow']

        vintage_strategy_fund = fund_info[
            (fund_info['VintageYear'] == vintage) &
            (fund_info['Strategy'] == strategy)
        ]['FundID']
        strat_cf = distr_df[distr_df['FundID'].isin(vintage_strategy_fund)].groupby('Quarter')['Cashflow'].sum()
        port_vintage_strategy_weight = port_distr_grouped[(port_distr_grouped['VintageYear'] == vintage) & (port_distr_grouped['Strategy'] == strategy)]['Cashflow'].values[0]
        strategy_cf += (vintage_weights_d[vintage]*port_vintage_strategy_weight / vintage_d_port[vintage] / weight) * strat_cf.reindex(np.arange(60), fill_value=0).values


    strategy_irr = npf.irr(strategy_cf)
    strategy_moic = moic(strategy_cf)

    # 4. GEOGRAPHY Alpha: allocation across North America vs Rest of World
    
    invested_grouped_g = (
        invested_df
        .groupby(['VintageYear', 'Geography'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )
    distr_grouped_g = (
        distr_df
        .groupby(['VintageYear', 'Geography'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )

    port_invested_grouped_g = (
        port_invested_df
        .groupby(['VintageYear', 'Geography'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )
    port_distr_grouped_g = (
        port_distr_df
        .groupby(['VintageYear', 'Geography'])['Cashflow']
        .sum()
        .abs()
        .reset_index()
    )

    geography_cf = np.zeros_like(market_cf)

    for idx, row in invested_grouped_g.iterrows():
        vintage = row['VintageYear']
        geo = row['Geography']
        weight = row['Cashflow']

        vintage_geo_fund = fund_info[
            (fund_info['VintageYear'] == vintage) &
            (fund_info['Geography'] == geo)
        ]['FundID']
        geo_cf = invested_df[invested_df['FundID'].isin(vintage_geo_fund)].groupby('Quarter')['Cashflow'].sum()
        port_vintage_geo_weight = port_invested_grouped_g[(port_invested_grouped_g['VintageYear'] == vintage) & (port_invested_grouped_g['Geography'] == geo)]['Cashflow'].values[0]
        geography_cf += (vintage_weights_c[vintage]*port_vintage_geo_weight / vintage_c_port[vintage] / weight) * geo_cf.reindex(np.arange(60), fill_value=0).values

    for idx, row in distr_grouped_g.iterrows():
        vintage = row['VintageYear']
        geo = row['Geography']
        weight = row['Cashflow']

        vintage_geo_fund = fund_info[
            (fund_info['VintageYear'] == vintage) &
            (fund_info['Geography'] == geo)
        ]['FundID']
        geo_cf = distr_df[distr_df['FundID'].isin(vintage_strategy_fund)].groupby('Quarter')['Cashflow'].sum()
        port_vintage_strategy_weight = port_distr_grouped_g[(port_distr_grouped_g['VintageYear'] == vintage) & (port_distr_grouped_g['Geography'] == geo)]['Cashflow'].values[0]
        geography_cf += (vintage_weights_d[vintage]*port_vintage_strategy_weight / vintage_d_port[vintage] / weight) * geo_cf.reindex(np.arange(60), fill_value=0).values


    geography_irr = npf.irr(geography_cf)
    geography_moic = moic(geography_cf)

    # 5. SIZING Alpha
    
    # Step 1: Calculate total invested capital (sum of negatives) for each fund
    invested_by_fund = (
        portfolio_df[portfolio_df['Cashflow'] < 0]
        .groupby('FundID')['Cashflow']
        .sum()
        .abs()
    )

    # Step 2: Map total invested capital back to each cash flow
    portfolio_df['total_invested'] = portfolio_df['FundID'].map(invested_by_fund)

    # Step 3: Divide each cash flow by total invested capital
    portfolio_df['new_cf'] = portfolio_df['Cashflow'] / portfolio_df['total_invested']
    eq_cf = portfolio_df.groupby('Quarter')['new_cf'].sum().sort_index().values
    sizing_irr = npf.irr(eq_cf)
    sizing_moic = moic(eq_cf)

    # Decompose
    timing_alpha = timing_irr - market_irr
    strategy_alpha = strategy_irr - market_irr
    geography_alpha = geography_irr - market_irr
    sizing_alpha = portfolio_irr - sizing_irr
    residual_alpha = portfolio_irr - (timing_alpha + strategy_alpha + geography_alpha + sizing_alpha + market_irr)

    return {
        'Market IRR': market_irr,
        'Portfolio IRR': portfolio_irr,
        'Timing Alpha': timing_alpha,
        'Strategy Alpha': strategy_alpha,
        'Geography Alpha': geography_alpha,
        'Sizing Alpha': sizing_alpha,
        'Residual Alpha': residual_alpha
    }


In [201]:
results = attribution_analysis(cashflows_df, portfolio_result)

for key, value in results.items():
    print(f"{key}: {value:.4%}")


Market IRR: 0.0051%
Portfolio IRR: 0.0082%
Timing Alpha: -0.0051%
Strategy Alpha: -0.0051%
Geography Alpha: 0.2909%
Sizing Alpha: -0.0868%
Residual Alpha: -0.1907%


C:\Users\Asus\AppData\Local\Temp\ipykernel_17912\3233728630.py:191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio_df['total_invested'] = portfolio_df['FundID'].map(invested_by_fund)
C:\Users\Asus\AppData\Local\Temp\ipykernel_17912\3233728630.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  portfolio_df['new_cf'] = portfolio_df['Cashflow'] / portfolio_df['total_invested']


In [202]:
portfolio_funds = portfolio_result['PortfolioFunds']
portfolio_cf = portfolio_result['Cashflows']
portfolio_irr = portfolio_result['IRR']
potrfolio_moic = portfolio_result['MOIC']
portfolio_df = portfolio_result['Portfolio_df']

# 1. MARKET Average IRR (baseline)
market_cf = cashflows_df.groupby('Quarter')['Cashflow'].sum().sort_index().values
market_irr = npf.irr(market_cf)
market_moic = moic(market_cf)

# 2. TIMING Alpha: invest according to portfolio vintages but into the market
# --> Weight vintages like the portfolio, but performance is market average
fund_info = cashflows_df[['FundID', 'VintageYear', 'Strategy', 'Geography']].drop_duplicates()

portfolio_info = fund_info[fund_info['FundID'].isin(portfolio_funds)]

# Get the market vintage distribution
invested_df = cashflows_df[cashflows_df['Cashflow'] < 0]
distr_df = cashflows_df[cashflows_df['Cashflow'] > 0]
vintage_c = invested_df.groupby('VintageYear')['Cashflow'].sum().abs()
vintage_d = distr_df.groupby('VintageYear')['Cashflow'].sum()
vintage_weights_c = invested_df.groupby('VintageYear')['Cashflow'].sum().abs() / abs(invested_df['Cashflow'].sum())
vintage_weights_d = distr_df.groupby('VintageYear')['Cashflow'].sum() / abs(distr_df['Cashflow'].sum())

#get the portfolio vintage distribution
port_invested_df = portfolio_df[portfolio_df['Cashflow'] < 0]
port_distr_df = portfolio_df[portfolio_df['Cashflow'] > 0]
vintage_c_port = port_invested_df.groupby('VintageYear')['Cashflow'].sum().abs()
vintage_d_port = port_distr_df.groupby('VintageYear')['Cashflow'].sum() 
vintage_weights_c_port = port_invested_df.groupby('VintageYear')['Cashflow'].sum().abs() / abs(port_invested_df['Cashflow'].sum())
vintage_weights_d_port = port_distr_df.groupby('VintageYear')['Cashflow'].sum() / abs(port_distr_df['Cashflow'].sum())

# Simulate market cash flows weighted by portfolio's vintage weights
timing_cf = np.zeros_like(market_cf)
for vintage, weight in vintage_weights_c_port.items():
    vintage_funds = fund_info[fund_info['VintageYear'] == vintage]['FundID']
    vintage_cf = invested_df[invested_df['FundID'].isin(vintage_funds)].groupby('Quarter')['Cashflow'].sum()
    timing_cf += (weight / vintage_c[vintage]) * vintage_cf.reindex(np.arange(60), fill_value=0).values

for vintage, weight in vintage_weights_d_port.items():
    vintage_funds = fund_info[fund_info['VintageYear'] == vintage]['FundID']
    vintage_cf = distr_df[distr_df['FundID'].isin(vintage_funds)].groupby('Quarter')['Cashflow'].sum()
    timing_cf += (weight / vintage_d[vintage]) * vintage_cf.reindex(np.arange(60), fill_value=0).values

timing_irr = npf.irr(timing_cf)
timing_moic = moic(timing_cf)